In [0]:
import glob
import os
from skimage.transform import resize
from numpy.linalg import norm
import cv2
import numpy as np
from skimage.io import imread, imshow
from skimage import exposure
import matplotlib.pyplot as plt
import pickle
import random
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

In [0]:
!wget https://www.dropbox.com/s/at3f7xuwec3xby3/dog_dataset.zip?dl=0 -O dog_dataset.zip
!unzip dog_dataset.zip
clear_output()

In [0]:
def get_data(path="./dog_dataset"):
    """
    :param path:
    :return: imgs - list of images
             nums - list of corresponding numerical breed-values
             dict{numerical breed: name of breed}
    """
    labels = os.listdir(path)
    labels.remove(".DS_Store")
    breeds = {}
    nums = []
    imgs = []
    for label in labels:
        label_path = path + "/" + label
        files = os.listdir(label_path)
        name = label[(label.find("-")+1):]
        num = int(label[1:label.find("-")])
        breeds[num] = name
        for file in files:
            file_path = label_path + "/" + file
            img = imread(file_path)
            imgs.append(img)
            nums.append(num)
    return imgs, nums, breeds

### SVM training

In [0]:
class StatModel(object):
    '''parent class - starting point to add abstraction'''
    def load(self, fn):
        self.model.load(fn)
    def save(self, fn):
        self.model.save(fn)

class SVM(StatModel):
    '''wrapper for OpenCV SimpleVectorMachine algorithm'''
    def __init__(self, kernel):
        self.model = cv2.ml.SVM_create()
        self.kernel = kernel

    def train(self, samples, responses):
        #setting algorithm parameters
        self.model.setType(cv2.ml.SVM_C_SVC)
        self.model.setKernel(f'cv2.ml.SVM_{self.kernel}')
        self.model.train(samples, cv2.ml.ROW_SAMPLE, responses)

    def predict(self, samples):
        res = []
        for sample in samples:
            pr = self.model.predict(sample)
            pr = pr[1][0][0]
            res.append(pr)
        return res

    def load(self, fn):
        self.model = cv2.ml.SVM_load(fn)

    def save(self, fn):
        self.model.save(fn)

In [0]:
from skimage.color import rgb2gray
from scipy.ndimage.filters import convolve

import SIFT.DoG_pyramid as pyramid
from SIFT.keypoints import get_keypoints
from SIFT.orientation import assign_orientation
from SIFT.descriptors import get_local_descriptors


class SIFT(object):
    def __init__(self, s=3, num_octave=4, s0=1.3, sigma=1.6, r_th=10, t_c=0.03, w=16, kernel='LINEAR'):
        self.s = s
        self.s0 = s0
        self.sigma = sigma
        self.num_octave = num_octave
        self.t_c = t_c
        self.R_th = (r_th+1)**2 / r_th
        self.w = w
        self.clf = SVM(kernel)

    def get_features(self, img):
        gaussian_pyr = pyramid.generate_gaussian_pyramid(img, self.num_octave, self.s, self.sigma)
        DoG_pyr = pyramid.generate_DoG_pyramid(gaussian_pyr)
        kp_pyr = get_keypoints(DoG_pyr, self.R_th, self.t_c, self.w)
        feats = []

        for i, DoG_octave in enumerate(DoG_pyr):
            kp_pyr[i] = assign_orientation(kp_pyr[i], DoG_octave)
            feats.append(get_local_descriptors(kp_pyr[i], DoG_octave))

        self.kp_pyr = kp_pyr
        self.feats = feats
        return feats

    def train(self, images, labels):
        descriptors = []
        count = 0
        for img in images:
            ft = np.float32(self.get_features(img))
            descriptors.append(ft)
            count += 1
        train_data = np.matrix(descriptors, dtype=np.float32)
        labels = np.array([labels]).transpose()
        self.clf.train(train_data, labels)

    def predict(self, samples):
        features = []
        for sample in samples:
            ft = self.get_features(sample)
            features.append(ft)
        predict_data = np.matrix(features, dtype=np.float32)
        return self.clf.predict(predict_data)

    def save(self, path):
        self.clf.save(path)

    def load(self, path):
        self.clf.load(path)

In [0]:
imgs, nums, breeds = get_data(path="./dog_dataset")

In [0]:
c = list(zip(imgs, nums))
random.shuffle(c)
imgs, nums = zip(*c) 2.83 1.09

In [0]:
half_len = int(0.5*len(nums))
imgs_train, imgs_test, nums_train, nums_test = train_test_split(imgs[:half_len], nums[:half_len], test_size=0.2, random_state=42)

In [0]:
def try_to_load_as_pickled_object_or_None(filepath):
    """
    This is a defensive way to write pickle.load, allowing for very large files on all platforms
    """
    max_bytes = 2**31 - 1
    try:
        input_size = os.path.getsize(filepath)
        print(input_size)
        bytes_in = bytearray(0)
        with open(filepath, 'rb') as f_in:
            for _ in range(0, input_size, max_bytes):
                bytes_in += f_in.read(max_bytes)
        obj = pickle.loads(bytes_in)
    except:
        return None
    return obj

In [0]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1eY0XYoQTJe0sPsmIGfwTCH5faLdocaJ5' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1eY0XYoQTJe0sPsmIGfwTCH5faLdocaJ5" -O imgs_test.pkl && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Nd5rVRjyIusrpa5BLiMtQVlxSQb9JFYv' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Nd5rVRjyIusrpa5BLiMtQVlxSQb9JFYv" -O imgs_train.pkl && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1HTDlQ4g4-KAW7T6BjzEz8DQ4asFGzvVr' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1HTDlQ4g4-KAW7T6BjzEz8DQ4asFGzvVr" -O nums_test.pkl && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1s5PSuD3RTtfUZTisSRC9Mhiq5c0mys3U' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1s5PSuD3RTtfUZTisSRC9Mhiq5c0mys3U" -O nums_train.pkl && rm -rf /tmp/cookies.txt
clear_output()

In [0]:
imgs_train = try_to_load_as_pickled_object_or_None("imgs_train.pkl")
imgs_test = try_to_load_as_pickled_object_or_None("imgs_test.pkl")
nums_train = try_to_load_as_pickled_object_or_None("nums_train.pkl")
nums_test = try_to_load_as_pickled_object_or_None("nums_test.pkl")

4464254113
1102425325
41039
10267


In [0]:
kernels = ["LINEAR", "RBF", "SIGMOID", "CHI2", "INTER"]

for kernel in kernels:
  sift = SIFT(kernel=kernel)
  sift.train(imgs_train, nums_train)
  sift.save(f'sift_svm_{kernel}_halfdata_train.dat')
  print(f"Completed {kernel}")

Completed LINEAR
Completed RBF
Completed SIGMOID
Completed CHI2
Completed INTER


In [0]:
for i, kernel in enumerate(kernels):
  sift.load(f'sift_svm_{kernel}_halfdata_train.dat')
  res = sift.predict(imgs_test)

  accuracy=0
  for i in range(len(res)):
    if res[i]==nums_test[i]:
      accuracy+=1
  accuracy = accuracy/len(res)

  print(f'Accuracy for SIFT with {kernel} kernel: {round(accuracy*100, 2)}')

Accuracy for SIFT with LINEAR kernel: 2.37
Accuracy for SIFT with RBF kernel: 2.61
Accuracy for SIFT with SIGMOID kernel: 1.84
Accuracy for SIFT with CHI2 kernel: 2.61
Accuracy for SIFT with INTER kernel: 4.52


### CatBoost training

In [0]:
sift = SIFT()
features_train = []
for img in imgs_train:
  features_train.append(np.float32(sift.get_features(img)))
features_train = np.array(features_train)

In [0]:
features_test = []
for img in imgs_test:
  features_test.append(np.float32(sift.get_features(img)))
features_test = np.array(features_test)

In [1]:
!pip install catboost

     |████████████████████████████████| 63.0MB 150kB/s 


In [0]:
from catboost import Pool, CatBoostClassifier

cat_features = [0]

train_dataset = Pool(data=features_train,
                     label=nums_train)

eval_dataset = Pool(data=features_test,
                    label=nums_test)

# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=500,
                           learning_rate=0.9,
                           depth=5,
                           task_type="GPU",
                           devices='0:1',
                           loss_function='MultiClass')
# Fit model
model.fit(train_dataset)
clear_output()

In [3]:
preds_class = model.predict(eval_dataset)

accuracy=0
for i in range(len(preds_class)):
  if preds_class[i]==nums_test[i]:
    accuracy+=1
accuracy = accuracy/len(nums_test)

print(f'Accuracy for CatBoost with LR = 0.9, iterations = 500, depth = 5: {round(accuracy*100, 2)}')

Accuracy for CatBoost with LR = 0.9, iterations = 500, depth = 5: 5.46


In [0]:
model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.9,
                           depth=2,
                           task_type="GPU",
                           devices='0:1',
                           loss_function='MultiClass')
# Fit model
model.fit(train_dataset)
clear_output()

In [5]:
preds_class = model.predict(eval_dataset)

accuracy=0
for i in range(len(preds_class)):
  if preds_class[i]==nums_test[i]:
    accuracy+=1
accuracy = accuracy/len(nums_test)

print(f'Accuracy for CatBoost with LR = 0.9, iterations = 1000, depth = 2: {round(accuracy*100, 2)}')

Accuracy for CatBoost with LR = 0.9, iterations = 1000, depth = 2: 6.14


In [0]:
model = CatBoostClassifier(iterations=5000,
                           learning_rate=0.5,
                           depth=2,
                           task_type="GPU",
                           devices='0:1',
                           loss_function='MultiClass')
# Fit model
model.fit(train_dataset)
clear_output()

In [8]:
preds_class = model.predict(eval_dataset)

accuracy=0
for i in range(len(preds_class)):
  if preds_class[i]==nums_test[i]:
    accuracy+=1
accuracy = accuracy/len(nums_test)

print(f'Accuracy for CatBoost with LR = 0.5, iterations = 5000, depth = 2: {round(accuracy*100, 2)}')

Accuracy for CatBoost with LR = 0.5, iterations = 5000, depth = 2: 7.23
